In [1]:
# https://adventofcode.com/2021/day/16
import numpy as np
import pandas as pd

In [2]:
def dec2hex(d):
    return format(d, 'x')

def hex2dec(h):
    return int(h, 16)

def dec2bin(d, n_digit=4):
    return format(d, '0' + str(n_digit) + 'b')

def hex2bin(h, n_digit=4):
    d = hex2dec(h)
    return dec2bin(d, n_digit=n_digit)

def bin2dec(b):
    return int(b, 2)

def binlist2dec(blist):
    return bin2dec(''.join(blist))


def read_packet_header(packet_bin):
    packet_version = bin2dec(''.join(packet_bin[:3]))
    type_id  = bin2dec(''.join(packet_bin[3:6]))
    contents = packet_bin[6:]
    
    return packet_version, type_id, contents


def read_n_bits(contents, n):
    return [contents.pop(0) for i in range(n)]


# contents will be updated.
def read_literal_value(contents):
    prefix = '1'
    numbers_bin = []
    n_bits = 0
    while prefix == '1':
        prefix = contents.pop(0)
        #numbers_bin.extend([contents.pop(0) for i in range(4)])
        numbers_bin.extend(read_n_bits(contents, 4))
        n_bits += 5
    return bin2dec(''.join(numbers_bin)), n_bits


def all_zero(contents):
    x = np.array(contents).astype(int)
    return np.all(x==0)

In [3]:
def read_packet(packet_bin, num=0, debug=False):
    '''
    num (int): counter about how many times this function is called.
    '''
    # packet_version and type_id are int.
    # contents are the list of char.
    n_bits = 6
    if all_zero(packet_bin):
        return -1, len(packet_bin), packet_bin
        
    packet_version, type_id, contents = read_packet_header(packet_bin)
    packet_versions.append(packet_version)
#     if debug:
#         print(f"packet version: {packet_version}")
#         print(f"type id: {type_id}")
    
    if type_id == 4: 
        # literal packet.
        value, n_bits_ = read_literal_value(contents)
        n_bits += n_bits_

    else:
        # operator packet.
        length_type_id = contents.pop(0)
#         if debug:
#             print(f"length type id: {length_type_id}")
        if length_type_id == '0':
            n_bits_subpackets = binlist2dec(read_n_bits(contents, 15))
#             if debug:
#                 print(f"n_bits subpacket: {n_bits_subpackets}")
        
            n_bits_total = 0
            while n_bits_total < n_bits_subpackets:
                value, n_bits, contents = read_packet(contents.copy(), num=num+1, debug=debug)
                n_bits_total += n_bits
                
        else:
            n_subpackets = binlist2dec(contents[:11])
            read_n_bits(contents, 11)

            n_subpackets_total = 0
            while n_subpackets_total < n_subpackets:
                value, n_bits, contents = read_packet(contents.copy(), num=num+1, debug=debug)
                n_subpackets_total += 1 
    
    if debug:
        print('=====')
        print(f"[{num}] type_id: {type_id}, value: {value} ({n_bits})")
    packet_values.append([num, type_id, value])
    return value, n_bits, contents


def calc_answer(type_id, values):
    if type_id == 0:
        answer = np.sum(values)
    elif type_id == 1:
        answer = np.prod(values)
    elif type_id == 2:
        answer = np.min(values)
    elif type_id == 3:
        answer = np.max(values)
    elif type_id == 4:
        answer = values[0]
    elif type_id == 5:
        answer = int(values[0]>values[1])
    elif type_id == 6:
        answer = int(values[0]<values[1])
    elif type_id == 7:
        answer = int(values[0]==values[1])
    return answer

In [29]:
## samples
# part 1.
#packet_hex = 'D2FE28'
#packet_hex = '38006F45291200'
#packet_hex = 'EE00D40C823060'
#packet_hex = '8A004A801A8002F478'
#packet_hex = '620080001611562C8802118E34'
#packet_hex = 'C0015000016115A2E0802F182340'
packet_hex = 'A0016C880162017C3686B18A3D4780'

## quiz input.
#packet_hex = 'E20D7880532D4E551A5791BD7B8C964C1548CB3EC1FCA41CC00C6D50024400C202A65C00C20257C008AF70024C00810039C00C3002D400A300258040F200D6040093002CC0084003FA52DB8134DE620EC01DECC4C8A5B55E204B6610189F87BDD3B30052C01493E2DC9F1724B3C1F8DC801E249E8D66C564715589BCCF08B23CA1A00039D35FD6AC5727801500260B8801F253D467BFF99C40182004223B4458D2600E42C82D07CC01D83F0521C180273D5C8EE802B29F7C9DA1DCACD1D802469FF57558D6A65372113005E4DB25CF8C0209B329D0D996C92605009A637D299AEF06622CE4F1D7560141A52BC6D91C73CD732153BF862F39BA49E6BA8C438C010E009AA6B75EF7EE53BBAC244933A48600B025AD7C074FEB901599A49808008398142013426BD06FA00D540010C87F0CA29880370E21D42294A6E3BCF0A080324A006824E3FCBE4A782E7F356A5006A587A56D3699CF2F4FD6DF60862600BF802F25B4E96BDD26049802333EB7DDB401795FC36BD26A860094E176006A0200FC4B8790B4001098A50A61748D2DEDDF4C6200F4B6FE1F1665BED44015ACC055802B23BD87C8EF61E600B4D6BAD5800AA4E5C8672E4E401D0CC89F802D298F6A317894C7B518BE4772013C2803710004261EC318B800084C7288509E56FD6430052482340128FB37286F9194EE3D31FA43BACAF2802B12A7B83E4017E4E755E801A2942A9FCE757093005A6D1F803561007A17C3B8EE0008442085D1E8C0109E3BC00CDE4BFED737A90DC97FDAE6F521B97B4619BE17CC01D94489E1C9623000F924A7C8C77EA61E6679F7398159DE7D84C015A0040670765D5A52D060200C92801CA8A531194E98DA3CCF8C8C017C00416703665A2141008CF34EF8019A080390962841C1007217C5587E60164F81C9A5CE0E4AA549223002E32BDCEA36B2E100A160008747D8B705C001098DB13A388803F1AE304600'


# convert hex2bin.
packet_bin = list(''.join([hex2bin(p) for p in packet_hex]))


## part 1.
packet_versions = []
packet_values = []
_ = read_packet(packet_bin, num=0, debug=True)
packet_values = np.array(packet_values[:-1])
print(f"packet_values: \n{packet_values}")
#print(f"sum of packet versions are {np.sum(packet_versions)}")

=====
[3] type_id: 4, value: 6 (11)
=====
[3] type_id: 4, value: 6 (11)
=====
[3] type_id: 4, value: 12 (11)
=====
[3] type_id: 4, value: 15 (11)
=====
[3] type_id: 4, value: 15 (11)
=====
[2] type_id: 0, value: 15 (11)
=====
[1] type_id: 0, value: 15 (11)
=====
[0] type_id: 0, value: -1 (7)
packet_values: 
[[ 3  4  6]
 [ 3  4  6]
 [ 3  4 12]
 [ 3  4 15]
 [ 3  4 15]
 [ 2  0 15]
 [ 1  0 15]]


In [6]:
# part 2.
#packet_hex = 'C200B40A82'
#packet_hex = '04005AC33890'
#packet_hex = '880086C3E88112'
#packet_hex = 'CE00C43D881120'
#packet_hex = 'D8005AC2A8F0'
#packet_hex = 'F600BC2D8F'
#packet_hex = '9C005AC2F8F0'
#packet_hex = '9C0141080250320F1802104A08'

## quiz input.
packet_hex = 'E20D7880532D4E551A5791BD7B8C964C1548CB3EC1FCA41CC00C6D50024400C202A65C00C20257C008AF70024C00810039C00C3002D400A300258040F200D6040093002CC0084003FA52DB8134DE620EC01DECC4C8A5B55E204B6610189F87BDD3B30052C01493E2DC9F1724B3C1F8DC801E249E8D66C564715589BCCF08B23CA1A00039D35FD6AC5727801500260B8801F253D467BFF99C40182004223B4458D2600E42C82D07CC01D83F0521C180273D5C8EE802B29F7C9DA1DCACD1D802469FF57558D6A65372113005E4DB25CF8C0209B329D0D996C92605009A637D299AEF06622CE4F1D7560141A52BC6D91C73CD732153BF862F39BA49E6BA8C438C010E009AA6B75EF7EE53BBAC244933A48600B025AD7C074FEB901599A49808008398142013426BD06FA00D540010C87F0CA29880370E21D42294A6E3BCF0A080324A006824E3FCBE4A782E7F356A5006A587A56D3699CF2F4FD6DF60862600BF802F25B4E96BDD26049802333EB7DDB401795FC36BD26A860094E176006A0200FC4B8790B4001098A50A61748D2DEDDF4C6200F4B6FE1F1665BED44015ACC055802B23BD87C8EF61E600B4D6BAD5800AA4E5C8672E4E401D0CC89F802D298F6A317894C7B518BE4772013C2803710004261EC318B800084C7288509E56FD6430052482340128FB37286F9194EE3D31FA43BACAF2802B12A7B83E4017E4E755E801A2942A9FCE757093005A6D1F803561007A17C3B8EE0008442085D1E8C0109E3BC00CDE4BFED737A90DC97FDAE6F521B97B4619BE17CC01D94489E1C9623000F924A7C8C77EA61E6679F7398159DE7D84C015A0040670765D5A52D060200C92801CA8A531194E98DA3CCF8C8C017C00416703665A2141008CF34EF8019A080390962841C1007217C5587E60164F81C9A5CE0E4AA549223002E32BDCEA36B2E100A160008747D8B705C001098DB13A388803F1AE304600'


# convert hex2bin.
packet_bin = list(''.join([hex2bin(p) for p in packet_hex]))

packet_version, type_id, contents = read_packet_header(packet_bin.copy())
print(f"type id: {type_id}")

packet_versions = []
packet_values = []
_ = read_packet(packet_bin, num=0, debug=False)
packet_values = np.array(packet_values[:-1])
#print(f"packet_values: \n{packet_values}")

type id: 0


In [12]:
n_ = packet_values[0][0]
values = []
for packet in packet_values:
    print(packet)
    n = packet[0]
    if n == 21:
#     if num == num_:
#         values.append(packet[2])
#     elif num == num_-1:
#         type_id = packet[1]
#         print(f"type_id: {type_id}, values: {values}")
#         print(f"answer: {calc_answer(type_id, values)}")
#         values = []
#     num = packet[0]

[     2      4 412760]
[          2           4 24720562325]
[  2   4 133]
[      2       4 1220231]
[   2    4 1038]
[   1    0 1038]
type_id: 0, values: [412760, 24720562325, 133, 1220231, 1038]
answer: 24722196487
[    21      4 289540]
[    20      2 289540]
[    19      1 289540]
[    18      1 289540]
[    17      0 289540]
[   17     4 47376]
[  18    4 2100]
[    18      4 435656]
[    18      4 430083]
[      18        4 15789783]
[      17        3 15789783]
[ 19   4 971]
[ 19   4 971]
[ 18   5 971]
[   18     4 10691]
[   17     1 10691]
[   16     1 10691]
[16  4 13]
[ 17   4 934]
[ 17   4 178]
[  17    4 6235]
[    17      4 758664]
[  17    4 7238]
[  16    2 7238]
[      17        4 11512421]
[ 17   4 896]
[ 16   0 896]
[17  4  2]
[16  2  2]
[        17          4 9564307436]
[        16          0 9564307436]
[        15          3 9564307436]
[        14          1 9564307436]
[        13          2 9564307436]
[        12          1 9564307436]
[        11          3 

In [18]:
#packet_values[packet_values[:, 2]!=-1]
packet_values

array([[           2,            4,       412760],
       [           2,            4,  24720562325],
       [           2,            4,          133],
       [           2,            4,      1220231],
       [           2,            4,         1038],
       [           1,            0,         1038],
       [          21,            4,       289540],
       [          20,            2,       289540],
       [          19,            1,       289540],
       [          18,            1,       289540],
       [          17,            0,       289540],
       [          17,            4,        47376],
       [          18,            4,         2100],
       [          18,            4,       435656],
       [          18,            4,       430083],
       [          18,            4,     15789783],
       [          17,            3,     15789783],
       [          19,            4,          971],
       [          19,            4,          971],
       [          18,          